#  Define Paths

In [3]:
import os

path = {
    "Train": os.path.join("E:\\Bee_Data\\Buzz 2\\Train_set"),
    "Val": os.path.join("E:\\Bee_Data\\Buzz 2\\Val_set"),
    "Test":os.path.join("E:\\Bee_Data\\Buzz 2\\Test_set"),     
}



# Import Libraries and Define Configuration Class

In [2]:
import os
import pandas as pd
import numpy as np
import librosa
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

class Config:
    def __init__(self, sampling_rate=44100, audio_duration=2, n_mfcc=40):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_mfcc = n_mfcc
        self.audio_length = int(self.sampling_rate * self.audio_duration)
        self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length / 512)), 1)


In [7]:
# Example usage:
import IPython.display as ipd
import wave
example_file = os.path.join(path['Train'], "None_swarming", "08_06_21_board2_Tha chua dang nhot","08_06_21_board2_Tha chua dang nhot_segment10.wav")
example_wav = wave.open(example_file)
print("Sampling (frame) rate = ", example_wav.getframerate())
print("Total samples (frames) = ", example_wav.getnframes())
print("Duration = ", example_wav.getnframes()/example_wav.getframerate())

ipd.Audio(example_file)

Sampling (frame) rate =  16000
Total samples (frames) =  32000
Duration =  2.0


In [ ]:
# Create a Config instance for all data
config = Config(sampling_rate=16000, audio_duration=2, n_mfcc=40)

# Define Function to Load and Process Audio

In [16]:
def load_and_process_audio(file_path, config):
    data, _ = librosa.core.load(file_path, sr=config.sampling_rate, res_type="kaiser_fast")

    if len(data) > config.audio_length:
        max_offset = len(data) - config.audio_length
        offset = np.random.randint(max_offset)
        data = data[offset:(config.audio_length + offset)]
    else:
        if config.audio_length > len(data):
            max_offset = config.audio_length - len(data)
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, config.audio_length - len(data) - offset), "constant")

    mfcc = librosa.feature.mfcc(y=data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
    mfcc_feature = np.hstack([np.mean(mfcc, axis=1), np.std(mfcc, axis=1)])

    return mfcc_feature

# Define Function to Prepare Data

In [17]:
def prepare_data(filenames, config, data_dir):
    X = []

    for filename in tqdm(filenames, total=len(filenames), desc='Processing Audio Files'):
        file_path = os.path.join(data_dir, filename)
        mfcc_feature = load_and_process_audio(file_path, config)
        X.append(mfcc_feature)

    return np.array(X)

# Process Example Data and Print Stats

In [8]:
example_data = [example_file]

# Create a example Config 
example_config = Config(sampling_rate=16000, audio_duration=2, n_mfcc=40)

# Process and print stats for the example data
example_processed_data = prepare_data(example_data, example_config, path["Train"])
print(f"Example data processed shape: {example_processed_data.shape}")

# Print the processed data for inspection
print("Example processed data:")
print(example_processed_data)

Processing Audio Files:   0%|          | 0/1 [00:00<?, ?it/s]

Example data processed shape: (1, 80)
Example processed data:
[[-5.4834967e+02  1.4200230e+02  3.3153938e+01  2.8572203e+01
   1.0203482e+01  6.5774016e+00  2.5833201e+00  7.1611538e+00
   8.1589451e+00  9.0433426e+00  6.2206883e+00  2.4263558e+00
   2.1297061e+00  2.8758876e+00 -3.4403741e-01  9.5191556e-01
  -2.9957232e+00 -2.4599977e+00 -1.2189020e+00 -1.3645297e+00
  -2.1363714e+00  1.0936688e+00 -2.1488838e+00 -1.2659619e+00
  -8.3267134e-01  5.1859510e-01  1.4951268e+00  4.9800375e-01
  -1.8886077e+00 -2.1255093e+00 -1.7854800e+00 -1.9079980e+00
  -2.8354800e+00 -2.0555441e+00 -1.7991379e+00 -1.7965283e+00
  -2.2729173e+00  1.0247298e-01 -1.0686605e+00 -2.8730577e-01
   1.2036632e+01  5.2677755e+00  6.7182660e+00  4.1284676e+00
   3.3821545e+00  4.1048102e+00  3.8637450e+00  3.9524875e+00
   2.8059390e+00  4.0050049e+00  3.9195495e+00  3.2085574e+00
   3.1580384e+00  3.6133385e+00  3.0242174e+00  3.0614781e+00
   3.0042832e+00  3.6599879e+00  2.9902687e+00  3.4301076e+00
   3.203

# Define Function to Get Audio File Paths

In [10]:
import os

# Function to get a list of all audio file paths in a directory and its subdirectories
def get_audio_file_paths(directory):
    audio_file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                audio_file_paths.append(os.path.join(root, file))
    return audio_file_paths

# Get Audio File Paths for Different Sets

In [11]:
none_swarming_train = get_audio_file_paths(os.path.join(path["Train"], "None_swarming"))
swarming_train = get_audio_file_paths(os.path.join(path["Train"], "Swarming"))

none_swarming_val = get_audio_file_paths(os.path.join(path["Val"], "None_swarming"))
swarming_val = get_audio_file_paths(os.path.join(path["Val"], "Swarming"))

none_swarming_test = get_audio_file_paths(os.path.join(path["Test"], "None_swarming"))
swarming_test = get_audio_file_paths(os.path.join(path["Test"], "Swarming"))

# Process and Print Stats for Different Datasets

In [12]:
import time

def process_and_print_stats(data, config, data_dir):
    t1 = time.time()
    processed_data = prepare_data(data, config, os.path.join(data_dir))
    t2 = time.time()

    return processed_data, t2 - t1

# Initialize total times for training, validation, and testing
total_time_training = 0
total_time_validation = 0
total_time_testing = 0

# Process and print stats for training, validation, and test datasets
Noneswarming_train, time_train = process_and_print_stats(none_swarming_train, config, path["Train"])
Swarming_train, time_swarming_train = process_and_print_stats(swarming_train, config, path["Train"])
total_time_training += time_train + time_swarming_train

Noneswarming_val, time_val = process_and_print_stats(none_swarming_val, config, path["Val"])
Swarming_val, time_swarming_val = process_and_print_stats(swarming_val, config, path["Val"])
total_time_validation += time_val + time_swarming_val

Noneswarming_test, time_test = process_and_print_stats(none_swarming_test, config, path["Test"])
Swarming_test, time_swarming_test = process_and_print_stats(swarming_test, config, path["Test"])
total_time_testing += time_test + time_swarming_test





Processing Audio Files:   0%|          | 0/12659 [00:00<?, ?it/s]

Processing Audio Files:   0%|          | 0/12073 [00:00<?, ?it/s]

Processing Audio Files:   0%|          | 0/2000 [00:00<?, ?it/s]

Processing Audio Files:   0%|          | 0/2249 [00:00<?, ?it/s]

Processing Audio Files:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing Audio Files:   0%|          | 0/6575 [00:00<?, ?it/s]

In [13]:
print(f"Total time elapsed for Training: {total_time_training:.2f} seconds")
print(f"Total time elapsed for Validation: {total_time_validation:.2f} seconds")
print(f"Total time elapsed for Testing: {total_time_testing:.2f} seconds")

Total time elapsed for Training: 366.22 seconds
Total time elapsed for Validation: 61.65 seconds
Total time elapsed for Testing: 228.59 seconds


In [14]:
total_count_none_swarming = len(none_swarming_train) + len(none_swarming_val) + len(none_swarming_test)
total_count_swarming = len(swarming_train) + len(swarming_val) + len(swarming_test)

print(f"Total number of 'None_swarming' files processed: {total_count_none_swarming}")
print(f"Total number of 'Swarming' files processed: {total_count_swarming}")
print(f"Overall total files processed: {total_count_none_swarming + total_count_swarming}")

Total number of 'None_swarming' files processed: 19659
Total number of 'Swarming' files processed: 20897
Overall total files processed: 40556


# Shapes of Processed Data

In [13]:
print("Noneswarming_train shape:", Noneswarming_train.shape)
print("Swarming_train shape:", Swarming_train.shape)

print("Noneswarming_val shape:", Noneswarming_val.shape)
print("Swarming_val shape:", Swarming_val.shape)

print("Noneswarming_test shape:", Noneswarming_test.shape)
print("Swarming_test shape:", Swarming_test.shape)

Noneswarming_train shape: (12659, 80)
Swarming_train shape: (12073, 80)
Noneswarming_val shape: (2000, 80)
Swarming_val shape: (2249, 80)
Noneswarming_test shape: (5000, 80)
Swarming_test shape: (6575, 80)


#  Save Extracted Features in Subdirectories

In [15]:
# Create the base directory
base_dir = "Features Extracted Data"
os.makedirs(base_dir, exist_ok=True)

# Create the subdirectories for MFCCs_Features, Train, Val, and Test
mfccs_folder = os.path.join(base_dir, "MFCCs_Features")
train_folder = os.path.join(mfccs_folder, "Train")
val_folder = os.path.join(mfccs_folder, "Val")
test_folder = os.path.join(mfccs_folder, "Test")

os.makedirs(mfccs_folder, exist_ok=True)
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Save extracted features in the Train folder
np.save(os.path.join(train_folder, 'Noneswarming_train'), Noneswarming_train)
np.save(os.path.join(train_folder, 'Swarming_train'), Swarming_train)

# Save extracted features in the Val folder
np.save(os.path.join(val_folder, 'Noneswarming_val'), Noneswarming_val)
np.save(os.path.join(val_folder, 'Swarming_val'), Swarming_val)

# Save extracted features in the Test folder
np.save(os.path.join(test_folder, 'Noneswarming_test'), Noneswarming_test)
np.save(os.path.join(test_folder, 'Swarming_test'), Swarming_test)
